In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [2]:
spark = SparkSession.builder.appName("classificationExample1").getOrCreate()
spark

# 타이타닉 데이터를 이용한 생존여부 예측 모델
## 로직스틱 회귀

In [3]:
# csv 파일 로드 
data = spark.read.csv('learning_spark_data/titanic.csv', header=True, inferSchema=True)
data.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [4]:
## 겵츨치 처리
from pyspark.sql.functions import col, sum,when,isnan
null_counts = data.select(
        [
                sum(when(col(c).isNull() | isnan(c), 1).otherwise(0)).alias(c)
                for c in data.columns
        ]
)
null_counts.show()

+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Gender|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|     0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+



In [5]:
# feature selection
data_1 = data.select('Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare')
data_1.show()

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     3|  male|NULL|    0|    0| 8.4583|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|


In [6]:
# age 결측치 처리 - 편균값으로 대체
mean_age = data_1.select('Age').agg({
    "Age":"mean" # age column에 mean 함수
}).collect()[0][0]
mean_age

29.69911764705882

In [7]:
data_1 = data_1.fillna({'Age': mean_age})
data_1.show()

+--------+------+------+-----------------+-----+-----+-------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|   Fare|
+--------+------+------+-----------------+-----+-----+-------+
|       0|     3|  male|             22.0|    1|    0|   7.25|
|       1|     1|female|             38.0|    1|    0|71.2833|
|       1|     3|female|             26.0|    0|    0|  7.925|
|       1|     1|female|             35.0|    1|    0|   53.1|
|       0|     3|  male|             35.0|    0|    0|   8.05|
|       0|     3|  male|29.69911764705882|    0|    0| 8.4583|
|       0|     1|  male|             54.0|    0|    0|51.8625|
|       0|     3|  male|              2.0|    3|    1| 21.075|
|       1|     3|female|             27.0|    0|    2|11.1333|
|       1|     2|female|             14.0|    1|    0|30.0708|
|       1|     3|female|              4.0|    1|    1|   16.7|
|       1|     1|female|             58.0|    0|    0|  26.55|
|       0|     3|  male|             20.0|    0|    0| 

In [8]:
# 데이터 인코딩 StringIndexer 
indexer = StringIndexer( inputCol='Gender', outputCol='SexIndexer')
data_1 = indexer.fit(data_1).transform(data_1)

In [9]:
data_1.show(5)

+--------+------+------+----+-----+-----+-------+----------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|SexIndexer|
+--------+------+------+----+-----+-----+-------+----------+
|       0|     3|  male|22.0|    1|    0|   7.25|       0.0|
|       1|     1|female|38.0|    1|    0|71.2833|       1.0|
|       1|     3|female|26.0|    0|    0|  7.925|       1.0|
|       1|     1|female|35.0|    1|    0|   53.1|       1.0|
|       0|     3|  male|35.0|    0|    0|   8.05|       0.0|
+--------+------+------+----+-----+-----+-------+----------+
only showing top 5 rows



In [13]:
# FeatureVector 생성
# VectorAssembler: 여러개의 feature 나열, 각 컬럼은 모델이 학습하는 데 사용됨
# outputCol: 결합된 feature 저장할 새로운 컬럼 이름 지정 
assembler = VectorAssembler(
    inputCols=['Pclass', 'SexIndexer', 'Age', 'SibSp', 'Parch', 'Fare'],
    outputCol='features'
)

data_1 = assembler.transform(data_1)
data_1.select('features', 'Survived').show(5)

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[3.0,0.0,22.0,1.0...|       0|
|[1.0,1.0,38.0,1.0...|       1|
|[3.0,1.0,26.0,0.0...|       1|
|[1.0,1.0,35.0,1.0...|       1|
|[3.0,0.0,35.0,0.0...|       0|
+--------------------+--------+
only showing top 5 rows



In [15]:
# 데이터셋 분활
train_data, test_data = data_1.randomSplit([0.8,0.2], seed=42)
train_data.show(5), test_data.show(5)

+--------+------+------+----+-----+-----+------+----------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|  Fare|SexIndexer|            features|
+--------+------+------+----+-----+-----+------+----------+--------------------+
|       0|     1|female| 2.0|    1|    2|151.55|       1.0|[1.0,1.0,2.0,1.0,...|
|       0|     1|female|25.0|    1|    2|151.55|       1.0|[1.0,1.0,25.0,1.0...|
|       0|     1|  male|18.0|    1|    0| 108.9|       0.0|[1.0,0.0,18.0,1.0...|
|       0|     1|  male|19.0|    1|    0|  53.1|       0.0|[1.0,0.0,19.0,1.0...|
|       0|     1|  male|19.0|    3|    2| 263.0|       0.0|[1.0,0.0,19.0,3.0...|
+--------+------+------+----+-----+-----+------+----------+--------------------+
only showing top 5 rows

+--------+------+------+-----------------+-----+-----+-------+----------+--------------------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|   Fare|SexIndexer|            features|
+--------+------+------+-----------------+-----+-----+--

(None, None)

In [21]:
# 모뎅 생성
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='features', labelCol='Survived')
lr_model = lr.fit(train_data)
predict = lr_model.transform(test_data)
predict.select('features', 'Survived', 'prediction').show(5)


+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|[1.0,1.0,50.0,0.0...|       0|       1.0|
|[1.0,0.0,21.0,0.0...|       0|       1.0|
|[1.0,0.0,24.0,0.0...|       0|       1.0|
|[1.0,0.0,29.0,0.0...|       0|       1.0|
|[1.0,0.0,29.69911...|       0|       1.0|
+--------------------+--------+----------+
only showing top 5 rows



In [23]:
predict.select('Survived', 'prediction').groupBy('Survived', 'prediction').count().show()

+--------+----------+-----+
|Survived|prediction|count|
+--------+----------+-----+
|       1|       0.0|   16|
|       0|       0.0|   72|
|       1|       1.0|   45|
|       0|       1.0|   12|
+--------+----------+-----+



In [27]:
from pyspark.sql.functions import expr
comp = predict.withColumn('correct', expr('case when Survived = prediction then 1 else 0 end'))
comp.where('correct=0').count()

28

In [31]:
# 틀린 데이터만 필터링 
predict.filter(col('Survived') != col('prediction')).show(10)

+--------+------+------+-----------------+-----+-----+--------+----------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|    Fare|SexIndexer|            features|       rawPrediction|         probability|prediction|
+--------+------+------+-----------------+-----+-----+--------+----------+--------------------+--------------------+--------------------+----------+
|       0|     1|female|             50.0|    0|    0| 28.7125|       1.0|[1.0,1.0,50.0,0.0...|[-1.9520233772457...|[0.12433289705143...|       1.0|
|       0|     1|  male|             21.0|    0|    1| 77.2875|       0.0|[1.0,0.0,21.0,0.0...|[-0.5063625084573...|[0.37604662481402...|       1.0|
|       0|     1|  male|             24.0|    0|    0|    79.2|       0.0|[1.0,0.0,24.0,0.0...|[-0.5000095386390...|[0.37753842718518...|       1.0|
|       0|     1|  male|             29.0|    0|    0|    30.0|       0.0|[1.0,0.0,29.0,0.0...|[-0.1615540

In [32]:
# 정확도 평가
comp.selectExpr('avg(correct) as accuracy').collect()[0]['accuracy']

0.8068965517241379

In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

eval = BinaryClassificationEvaluator(labelCol='Survived',
                                    rawPredictionCol='rawPrediction',
                                    metricName='areaUnderROC')

auc = eval.evaluate(predict)
auc

0.8664129586260734

In [34]:
# AUROC -> X축 FPR, y축 TPR의 곡선 아래 면적, 1에 가까울수록 좋은 모델 

In [35]:
spark.stop()

#libsvm 차일 형식의 처리
텍스트 파일 형식, 희소데이터용 압축 파일
진짜 가끔 1이 나오는 애들 -> 희소 데이터
0인 애들은  아예 무시, 1인 애들만 저장 -> 데이터로 압축   
- 파일 사이즈가 그렇게 크지 않음 -> 처리속도 개선 - 머신러닝

레이블 행:값 행: 값 

In [36]:
spark = SparkSession.builder.appName("classificationExample2").getOrCreate()
spark

In [43]:
# 데이터 불러오기
data2 = spark.read.format('libsvm').load('learning_spark_data/sample_libsvm_data.txt')
data2

DataFrame[label: double, features: vector]

In [44]:
data2.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 10 rows



In [46]:
train_data, test_data = data2.randomSplit([0.7, 0.3], seed=12)
train_data.show(5), test_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[95,96,97,12...|
|  0.0|(692,[121,122,123...|
|  0.0|(692,[122,123,124...|
|  0.0|(692,[122,123,148...|
|  0.0|(692,[123,124,125...|
+-----+--------------------+
only showing top 5 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[98,99,100,1...|
|  0.0|(692,[100,101,102...|
|  0.0|(692,[123,124,125...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
+-----+--------------------+
only showing top 5 rows



(None, None)

In [48]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train_data)
pred = lrModel.transform(test_data)
pred.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[0.95414441393043...|[0.72194788886091...|       0.0|
|  0.0|(692,[100,101,102...|[0.48568919283978...|[0.61909039185191...|       0.0|
|  0.0|(692,[123,124,125...|[1.00961478127974...|[0.73294475454830...|       0.0|
|  0.0|(692,[126,127,128...|[0.90293696106823...|[0.71155267489794...|       0.0|
|  0.0|(692,[126,127,128...|[0.75830339388632...|[0.68098526895593...|       0.0|
|  0.0|(692,[127,128,129...|[0.97194839391749...|[0.72550768363690...|       0.0|
|  0.0|(692,[127,128,129...|[0.88414239930651...|[0.70767989197633...|       0.0|
|  0.0|(692,[127,128,129...|[0.96585465524705...|[0.72429247205628...|       0.0|
|  0.0|(692,[128,129,130...|[0.91063904393044...|[0.71313091310478...|       0.0|
|  0.0|(692,[152

In [50]:
# 평가 
eval = BinaryClassificationEvaluator(metricName='areaUnderROC')
auc = eval.evaluate(pred)
auc

1.0

In [51]:
spark.stop()

In [ ]:
# 문류모델 실습 완료 